In [1]:
from entsoe import EntsoePandasClient
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

client = EntsoePandasClient(api_key="b18dfce9-f1e3-4d07-822f-4abd1438e602")

In [2]:
start = pd.Timestamp('20220901', tz='Europe/Amsterdam')
end = pd.Timestamp('20230228', tz='Europe/Amsterdam')

countries = ['AT', 'BE', 'CZ', 'DE_LU', 'FR', 'HR', 'HU', 'NL', 'PL', 'RO', 'SI', 'SK']
hubs = ['ALBE', 'ALDE', 'AT', 'BE', 'CZ', 'DE_LU', 'FR', 'HR', 'HU', 'NL', 'PL', 'RO', 'SI', 'SK']

In [3]:
df_ren_gen = pd.DataFrame()

for country_code in countries:
    df_gen = client.query_wind_and_solar_forecast(country_code, start=start,end=end)
    if 'Solar' not in df_gen:
        df_gen['Solar'] = 0
    if 'Wind Onshore' not in df_gen:
        df_gen['Wind Onshore'] = 0
    if 'Wind Offshore' not in df_gen:
        df_gen['Wind Offshore'] = 0

    df_gen = df_gen.tz_localize(None)
    df_gen = df_gen.resample("H").agg(["mean"])

    df_ren_gen[country_code] = df_gen['Solar'] + df_gen['Wind Onshore'] + df_gen['Wind Offshore']
    
df_ren_gen['ALBE'] = 0
df_ren_gen['ALDE'] = 0

df_ren_gen = df_ren_gen[hubs]

In [4]:
start_autumn_test = np.datetime64("2022-11-01")
end_autumn_test = np.datetime64("2022-11-30")

start_winter_test = np.datetime64("2023-02-01")
end_winter_test = np.datetime64("2023-02-28")

def validation_mask(df):
    return ((df.index >= start_autumn_test) & (df.index <= end_autumn_test)) | ((df.index >= start_winter_test) & (df.index <= end_winter_test))

In [5]:
df_ren_gen.to_excel("./optimisation/data/renewable_generation.xlsx")
df_ren_gen_test = df_ren_gen.loc[validation_mask(df_gen)]
display(df_ren_gen_test)
df_ren_gen_test.to_excel("./optimisation/data/validation/renewable_generation.xlsx")

,ALBE,ALDE,AT,BE,CZ,DE_LU,FR,HR,HU,NL,PL,RO,SI,SK
2022-11-01 00:00:00,0,0,1102.0,3578.0,0.0,21395.75,12688.0,49.0,65.75,5246.25,428.0,436.50,0.0,1.0
2022-11-01 01:00:00,0,0,1104.0,3705.0,0.0,22046.00,12991.0,51.0,72.50,5482.50,483.0,277.50,0.0,1.0
2022-11-01 02:00:00,0,0,1043.0,3857.0,0.0,22912.50,12943.0,56.0,76.00,5822.50,561.0,194.00,0.0,1.0
2022-11-01 03:00:00,0,0,915.0,3966.0,0.0,23395.50,12895.0,56.0,76.00,6117.25,633.0,127.75,0.0,1.0
2022-11-01 04:00:00,0,0,762.0,3909.0,0.0,24454.50,12849.0,54.0,72.75,6412.00,736.0,75.00,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-27 19:00:00,0,0,1050.0,2280.0,0.0,9591.25,10121.0,620.0,30.50,2536.75,577.0,825.25,0.0,1.0
2023-02-27 20:00:00,0,0,980.0,2271.0,0.0,9460.25,9626.0,615.0,31.50,2377.00,529.0,856.00,0.0,1.0
2023-02-27 21:00:00,0,0,864.0,2203.0,0.0,9066.75,9130.0,605.0,28.50,2209.00,465.0,892.50,0.0,1.0
2023-02-27 22:00:00,0,0,742.0,2111.0,0.0,8499.00,8634.0,601.0,24.50,1925.25,387.0,899.50,0.0,1.0
